# Cohere 재랭킹 실험

https://cohere.com/rerank

https://docs.cohere.com/docs/rerank

https://python.langchain.com/docs/integrations/retrievers/cohere-reranker/

Cohere(캐나다)는 기업용 AI, 자연어 처리, 검색 및 생성형 AI 분야에서 강력한 솔루션을 제공하는 플랫폼이다.

Command R, Embed, Rerank와 같은 모델을 통해, 문서 검색, 요약, 질의응답, 다국어 지원 등 다양한 업무 자동화 및 데이터 활용이 가능하다. 또한 API와 다양한 클라우드 환경 지원으로, 개발자와 기업이 쉽게 도입할 수 있는 것이 큰 장점이다.

## Bi-Encoder와 Cross-Encoder 비교

**Bi-Encoder**와 **Cross-Encoder**는 문장 쌍의 관계(예: 유사도, 연관성 등)를 계산할 때 사용하는 대표적인 두 가지 구조이다. 각각의 구조와 특징을 간단하게 비교하면 다음과 같다.

| 구조         | 입력 방식                                      | 연산 속도         | 성능(정확도)         | 특징 요약                        |
|:------------:|:---------------------------------------------:|:----------------:|:--------------------:|:-------------------------------:|
| **Bi-Encoder**   | 두 문장을 각각 독립적으로 임베딩                | 빠름              | 다소 낮음             | 임베딩 미리 계산/저장 가능, 대량 비교 적합 |
| **Cross-Encoder**| 두 문장을 [SEP]으로 연결해 한 번에 입력           | 느림              | 높음                  | 문장 간 상호작용 정보 최대 활용, 소규모 비교 적합 |

**설명**

- **Bi-Encoder**  
  - 두 문장을 각각 독립적으로 인코더(BERT 등)에 넣어 임베딩 벡터를 만든다.
  - 만들어진 임베딩 벡터끼리 코사인 유사도 등으로 비교한다.
  - 임베딩을 미리 계산해 둘 수 있으므로, 대규모 문장 비교에서 매우 빠른 속도를 낼 수 있다.
  - 하지만 문장 간의 미세한 상호작용 정보가 손실될 수 있어, Cross-Encoder에 비해 정확도가 낮다.

- **Cross-Encoder**  
  - 두 문장을 [SEP] 토큰으로 연결해 한 번에 인코더에 넣는다.
  - 모델이 두 문장 사이의 상호작용 정보를 직접 활용해 결과(유사도 등)를 바로 출력한다.
  - 모든 문장 쌍마다 모델 연산이 필요하므로, 비교해야 할 문장이 많아질수록 속도가 매우 느려진다.
  - 하지만 문장 간 관계를 더욱 정확하게 파악할 수 있어, 성능(정확도)이 높다.

**정리**
- **Bi-Encoder**는 속도가 빠르지만, 성능(정확도)은 Cross-Encoder보다 낮다.
- **Cross-Encoder**는 성능이 뛰어나지만, 연산량이 많아 속도가 느리다.
- 실제로는 Bi-Encoder로 후보군을 먼저 빠르게 좁히고, Cross-Encoder로 최종 순위를 정하는 식으로 두 구조를 조합해 사용하는 경우가 많다.

**수식 예시**  
- Bi-Encoder에서 문장 임베딩 $u$, $v$를 얻고, 코사인 유사도는 다음과 같이 계산한다:
  $$
  \text{CosineSimilarity}(u, v) = \frac{u \cdot v}{\|u\|\|v\|}
  $$
  이 연산은 임베딩만 있으면 매우 빠르게 수행된다.

- Cross-Encoder는 두 문장을 [SEP]으로 연결해 입력한 뒤, 모델의 출력(예: [CLS] 토큰)을 통해 유사도를 바로 얻는다.




In [ ]:
%pip install -Uq python-dotenv langchain langchain-openai langchain-pinecone langchain-cohere kiwipiepy rank_bm25 scikit-learn

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")
os.environ['COHERE_API_KEY'] = os.getenv('cohere_key')

## 데이터 준비

In [ ]:
import pandas as pd

documents_df = pd.read_csv('documents.csv')
queries_df = pd.read_csv('queries.csv')

## BM25

In [ ]:
# BM25 키워드 검색 모델
# - 문서를 토큰화해서 BM25모델 생성
# - 검색어도 동일하게 토큰화한 이후 BM25 검색

from rank_bm25 import BM25Okapi
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenize(doc):
    return [token.form for token in kiwi.tokenize(doc)]

tokenized_docs = [kiwi_tokenize(doc) for doc in documents_df['content']]
bm25 = BM25Okapi(tokenized_docs)

def bm25_search(query, top_k=5):
    query_tokens = kiwi_tokenize(query)
    # print(query_tokens)
    scores = bm25.get_scores(query_tokens) # 30개의 문서에 대한 bm25 점수
    # print(scores)

    ranked_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    retrieved_docs = [documents_df['doc_id'].iloc[i] for i in ranked_idx[:top_k]]
    return retrieved_docs

bm25_search('제주도 관광 명소')

## 벡터 서치

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding_model = OpenAIEmbeddings()
vector_store = PineconeVectorStore(
    index_name='ir',
    embedding=embedding_model
 )

## cohere reranking

In [ ]:
# reranking 후보군선정
bm25_candidates = {}   # BM23 후보 저장용 dict
dense_candidates = {}  # Dense 후보 저장용 dict

for idx, row in queries_df.iterrows():
    qid = row['query_id']
    query_text = row['query_text']
    bm25_candidates[qid] = bm25_search(query_text, top_k=20)  # BM25 상위 20개 doc_id 후보
    docs = vector_store.similarity_search(query_text, k=20)   # 벡터 검색 상위 20개 문서
    dense_candidates[qid] = [doc.metadata['doc_id'] for doc in docs]  # doc_id만 추출

In [ ]:
qid = 'Q1'
merged_candidates = bm25_candidates[qid] + dense_candidates[qid]  # 두 후보 리스트 합치기
merged_candidates = list(set(merged_candidates))  # 중복 제거 후 리스트 재변환
print(len(merged_candidates))  # 병합 후 후보 개수
print(merged_candidates)  # 병합된 후보 doc_id 리스트

In [ ]:
# cohere reranking api 호출함수
from cohere import Client, TooManyRequestsError
from tqdm.auto import tqdm
import time

# cohere 클라이언트
co = Client()

# 쿼리와 후보 문서를 입력받아 Cohere rerank 결과를 반환하는 함수
def cohere_rerank(query, candidates, max_retries=1, wait_seconds=10):
    # 문서 내용가져오기
    texts = [
        documents_df.loc[documents_df['doc_id'] == doc_id, 'content'].values[0]  # doc_id로 본문 조회
            for doc_id in candidates
    ]

    def call_rerank_api():
        # Cohere rerank API 호출
        response = co.rerank(
            model='rerank-multilingual-v3.0',  # 다국어 rerank 모델
            query=query,      # 사용자 쿼리
            documents=texts   # 후보 문서 텍스트
        )
        # relevance_score 기준 내림차순 정렬
        rerank_index = sorted(response.results, key=lambda x: x.relevance_score, reverse=True)
        return [candidates[r.index] for r in rerank_index]  # 재정렬된 doc_id 반환

    for attempt in range(max_retries + 1):  # 재시도 횟수만큼 반복
        try:
            return call_rerank_api()  # API 호출 시도
        except TooManyRequestsError:
            if attempt < max_retries:
                print(
                    f'TooManyRequestsError: {attempt + 1} / {max_retries + 1} '
                    f'{wait_seconds}초후 다시 시도합니다.'
                )
                time.sleep(wait_seconds)
            else:
                raise # 예외 전파

cohere_rerank('제주도 관광명소', ['D26', 'D16', 'D25', 'D3', 'D4', 'D7', 'D30', 'D18', 'D2', 'D9', 'D6', 'D13', 'D15', 'D14', 'D17', 'D20', 'D8', 'D11', 'D10', 'D1', 'D12', 'D5', 'D19'])

### cohere_rerank함수 부분검증

In [ ]:
qid = 'Q1'
merged_candidates = bm25_candidates[qid] + dense_candidates[qid]
merged_candidates = list(set(merged_candidates))
print(len(merged_candidates))
print(merged_candidates)

In [ ]:
print(cohere_rerank('제주도 관광명소', merged_candidates))

In [ ]:
# doc_id -> content 조회
texts = [
    documents_df.loc[documents_df['doc_id'] == doc_id, 'content'].values[0]
        for doc_id in merged_candidates
]

In [ ]:
documents_df.loc[documents_df['doc_id'] == 'D26', 'content'].values[0]

In [ ]:
# Cohere Rerank 응답 확인 및 doc_id 재정렬 출력
from pprint import pprint
query = '제주도 관광 명소'
response = co.rerank(
    model='rerank-multilingual-v3.0',
    query=query,
    documents=texts
)
pprint(response.results)  # (문서별 점수/ 인덱스 등) 출력
rerank_index = sorted(response.results, key=lambda x: x.relevance_score, reverse=True)
print([merged_candidates[r.index] for r in rerank_index])  # 정렬된 순서대로 doc_id 리스트 출력

In [ ]:
# cohere rerank 호출
rerank_results = {}
top_k = 5

for idx, row in tqdm(queries_df.iterrows()):
    qid = row['query_id']
    query_text = row['query_text']

    merged_candidates = list(set(bm25_candidates[qid] + dense_candidates[qid]))
    rerank_results[qid] = cohere_rerank(query_text, merged_candidates)[:top_k]  # rerank 후 상위 5개 저장
    time.sleep(6)

rerank_results

## 검색성능 평가

**평가지표 설명**

* **Precision\@k**: 상위 k개의 검색 결과 중에 진짜 필요한 문서가 얼마나 있는지를 측정한다.
  - 예컨대 k=5일 때, 결과 5개 중 관련 문서가 2개면 Precision\@5 = 2/5 = 0.4다.

* **Recall\@k**: 전체 관련 문서 중에서 상위 k개 안에 얼마나 많이 들어왔는지를 본다.
  - 예컨대 전체 관련 문서가 4개이고, 그중 3개가 상위 5개 안에 들어오면 Recall\@5 = 3/4 = 0.75다.


* **MRR (Mean Reciprocal Rank)**:
  사용자가 제시한 여러 질의에서, 각 질의별로 “첫 번째 관련 문서”가 나온 순위의 역수를 구한 뒤 평균낸 것이다.
  **예시**

  * 질의 A: 첫 관련 문서가 2위 → RR = 1/2 = 0.5
  * 질의 B: 첫 관련 문서가 3위 → RR = 1/3 ≈ 0.333
  * 질의 C: 첫 관련 문서가 1위 → RR = 1/1 = 1
  * 이 세 질의의 MRR = (0.5 + 0.333 + 1) / 3 ≈ 0.611

* **AP (Average Precision)**:
  한 질의 결과 리스트를 순서대로 훑어가며, 관련 문서를 만날 때마다 그 시점까지의 Precision을 계산한 뒤, 관련 문서 개수로 나눈 값이다.
  **예시** (관련 문서 3개가 있고, 순위 2, 4, 5위에 위치한 경우)

  1. 2위에서 첫 관련 문서 발견 → Precision\@2 = 1/2 = 0.50
  2. 4위에서 두 번째 관련 문서 발견 → Precision\@4 = 2/4 = 0.50
  3. 5위에서 세 번째 관련 문서 발견 → Precision\@5 = 3/5 = 0.60
     AP = (0.50 + 0.50 + 0.60) / 3 ≈ 0.533

In [ ]:
import numpy as np

# 참조문서 답안 파싱
def parse_relevant(relevant_str) -> dict[str, int]:
    """
    relevant_str = 'D1=3;D4=1;D30=1' -> {'D1': 3, 'D4': 1, 'D30': 1}
    """
    pairs = relevant_str.split(';')
    rel_dict = {}
    for pair in pairs:
        doc_id, grade = pair.split('=')
        rel_dict[doc_id] = int(grade)
    return rel_dict


# 평가지표 계산
def compute_metrics(predicted, relevant_dict, k=5) -> tuple[float, float, float, float]:
    # Precision@k
    hits = sum([1 for doc in predicted[:k] if doc in relevant_dict])
    precision = hits / k

    # Recall@k
    total_relevant = len(relevant_dict) # {'D1': 3, 'D12': 2, ...}
    recall = hits / total_relevant if total_relevant > 0 else 0

    # MRR 예측치중 첫 관련문서 순위점수
    rr = 0
    for idx, doc in enumerate(predicted):
        if doc in relevant_dict:
            rr = 1 / (idx + 1)
            break

    # AP(MAP를 위한 계산)
    num_correct = 0
    precisions = []
    for idx, doc in enumerate(predicted[:k]):
        if doc in relevant_dict:
            num_correct += 1
            precisions.append(num_correct / (idx + 1))
    ap = np.mean(precisions) if precisions else 0
    return precision, recall, rr, ap


# 평가함수
def evaluate_all(method_results, queries_df, k=5):
    prec_list, rec_list, rr_list, ap_list = [], [], [], []

    for idx, row in queries_df.iterrows():
        qid = row['query_id']
        relevant_dict = parse_relevant(row['relevant_doc_ids'])
        predicted = method_results[qid]
        p, r, rr, ap = compute_metrics(predicted, relevant_dict, k)
        prec_list.append(p)
        rec_list.append(r)
        rr_list.append(rr)
        ap_list.append(ap)

    return {
        'P@k': np.mean(prec_list),
        'R@k': np.mean(rec_list),
        'MRR': np.mean(rr_list),
        'MAP': np.mean(ap_list),
    }


## 성능평가

In [ ]:
bm25_results = {qid:doc_ids[:5] for qid, doc_ids in bm25_candidates.items()}
dense_results = {qid:doc_ids[:5] for qid, doc_ids in dense_candidates.items()}

bm25_metrics = evaluate_all(bm25_results, queries_df, k=5)
dense_metrics = evaluate_all(dense_results, queries_df, k=5)
rerank_metrics = evaluate_all(rerank_results, queries_df, k=5)

In [ ]:
import pandas as pd
metrics_df = pd.DataFrame({
    'Metrics': ['P@5', 'R@5', 'MRR', 'MAP'],
    'BM25': [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']],
    'Dense': [dense_metrics['P@k'], dense_metrics['R@k'], dense_metrics['MRR'], dense_metrics['MAP']],
    'ReRank': [rerank_metrics['P@k'], rerank_metrics['R@k'], rerank_metrics['MRR'], rerank_metrics['MAP']],
})
metrics_df


In [ ]:
# 시각화
import matplotlib.pyplot as plt

metrics = ['P@5', 'R@5', 'MRR', 'MAP']
bm25_vals = [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']]
dense_vals = [dense_metrics['P@k'], dense_metrics['R@k'], dense_metrics['MRR'], dense_metrics['MAP']]
rerank_vals = [rerank_metrics['P@k'], rerank_metrics['R@k'], rerank_metrics['MRR'], rerank_metrics['MAP']]

x = range(len(metrics))

plt.figure(figsize=(8, 4))
plt.plot(x, bm25_vals, marker='o', label='BM25')
plt.plot(x, dense_vals, marker='s', label='Dense')
plt.plot(x, rerank_vals, marker='s', label='ReRank')
plt.xticks(x, metrics)
plt.ylim(0, 1.1)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('BM25 vs Dense Retrieval vs ReRank')
plt.legend()
plt.grid()
plt.show()

### ReRank
- 전반적 성능 최고
    - ReRank는 P@5, R@5, MRR, MAP 전 지표에서 가장 안정적으로 높거나 최고 수준을 기록함.
- 초기 검색 한계 보완
    - BM25/Dense가 만든 후보군을 의미 기반으로 재정렬하면서, 상위 랭크의 품질이 확실히 개선됨.
- 특히 강한 지표
    - MRR / MAP에서 ReRank가 가장 높음 → 첫 정답을 더 앞에 배치하고 전반적인 랭킹 품질이 우수함.
    - R@5도 Dense와 비슷하거나 더 높아, 정답 회수력을 유지하면서 정렬 품질까지 향상.
- 결론적으로 BM25 + Dense로 후보를 넓게 뽑고, ReRank로 최종 정렬하는 파이프라인이 가장 효과적


##### 일반적인 사용법
1. 1차 검색 (Recall 단계)
- BM25
    - 장점: 빠름, 비용 0, 안정적
    - 역할: “일단 후보를 최대한 안 놓치고 긁어온다”
- Dense Retrieval (Embedding)
    - 장점: 의미 검색 강함
    - 역할: 키워드가 달라도 의미적으로 관련 문서 확보  
-> Top 50 ~ 200개 후보 정도 확보 (이 단계에서는 순서 중요 ❌, 포함 여부가 중요 ⭕)

2. 후보 병합 (Recall 보장)
- BM25 ∪ Dense 결과 union
- 중복 제거만 수행  
-> “정답이 여기 안에만 있으면 성공” 이라는 단계

3. ReRank (Precision 단계)
- Cohere / Cross-Encoder / LLM Rerank
- 장점:
    - 문서 전체 문맥을 보고 정렬
    - MRR / MAP 급상승
- 단점:
    - 비쌈
    - 느림 (API / GPU)  
-> Top 5 ~ 10개만 최종 노출